In [1]:
from typing import Callable
from time import time
import pandas as pd

# for neural networks
from keras.models import Sequential
from keras.layers import Dense, Dropout
import tensorflow as tf

# for evaluation & preprocessing
from sklearn.model_selection import (
    train_test_split,
    ParameterGrid,
)
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
)
import sys, os
sys.path.append(os.path.abspath(os.path.join("..")))

# for displaying results & feedback
# from tabulate import tabulate
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [2]:
import sys, os

sys.path.append(os.path.abspath('..'))
%load_ext autoreload
%autoreload 2
from modules.config import *

In [3]:
model_data = pd.read_pickle(MODEL_DATA_PATH)
model_data.head(2)

,demand,hour,day,weekday,month,sustenance_poi_start,public_transport_poi_start,education_poi_start,arts_and_culture_poi_start,sports_poi_start,...,end_881f1abb2dfffff,end_881f1abb31fffff,end_881f1abb35fffff,end_881f1abb39fffff,end_881f1abb61fffff,end_881f1abb63fffff,end_881f1abb65fffff,end_881f1abb67fffff,end_881f1abb69fffff,end_881f1abb6bfffff
0,3,0,20,6,1,82,80,11,5,5,...,0,0,0,0,0,0,0,0,0,0
1,1,12,30,2,1,82,80,11,5,5,...,0,0,0,0,0,0,0,0,0,0


In [4]:
y = model_data["demand"]
X = model_data.drop(columns=["demand"])

X_train, X_rest, y_train, y_rest = train_test_split(X, y, train_size=0.7, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_rest, y_rest, test_size=0.5, random_state=42)

print(f"Size of the train dataset is: {X_train.shape[0]}")
print(f"Size of the validation dataset is: {X_valid.shape[0]}")
print(f"Size of the test dataset is: {X_test.shape[0]}")

Size of the train dataset is: 260545
Size of the validation dataset is: 55831
Size of the test dataset is: 55832


In [5]:
X_train = StandardScaler().fit_transform(X_train)
X_valid = StandardScaler().fit_transform(X_valid)
X_test = StandardScaler().fit_transform(X_test)

In [6]:
n_features = X_train.shape[1]

In [7]:
def mean_average_percentage_error(y_true, y_pred):
    return mean_absolute_error(y_true, y_pred) / y_true.mean()


def root_mean_squared_error(y_true, y_pred):
    return mean_squared_error(y_true, y_pred) ** 0.5

In [8]:
def train_model(n_nodes, n_layers, activation):
    model = Sequential()
    model.add(Dense(n_nodes, activation=activation, input_shape=(n_features,)))
    for _ in range(n_layers - 1):
        model.add(Dense(n_nodes, activation=activation))
    model.add(Dense(1, activation="relu"))

    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

    # measure time
    start = time()
    model.fit(X_train, y_train, epochs=20, batch_size=500)
    end = time()

    return model, start, end

In [9]:
# Hyperparameters:
# number of layers
n_layers_options = [1, 2, 3] #n_layers = [1, 2, 3, 4, 5]
# number of nodes
n_nodes_options = [n_features*0.5, n_features, n_features*1.5]
# learning rate
# momentum

# Other things to consider:
# activation function
activation_options = ["tanh", "sigmoid", "relu"]
# epochs
# epochs = [100, 200, 300]
# batch size
# batch_size = [128, 256, 512]

In [10]:
results = []

for n_nodes in n_nodes_options:
    for n_layers in n_layers_options:
        for activation in activation_options:
            model, start, end = train_model(n_nodes, n_layers, activation)
            y_pred = model.predict(X_test)

            results.append([
                n_layers,
                n_nodes,
                activation,
                end - start,
                mean_squared_error(y_test, y_pred),
                mean_absolute_error(y_test, y_pred),
                mean_average_percentage_error(y_test, y_pred),
                root_mean_squared_error(y_test, y_pred),
            ])

Epoch 1/20
522/522 [==============================] - 4s 6ms/step - loss: 1.4941 - mae: 0.8150
Epoch 2/20
522/522 [==============================] - 3s 6ms/step - loss: 1.3151 - mae: 0.7485
Epoch 3/20
522/522 [==============================] - 3s 6ms/step - loss: 1.2222 - mae: 0.7176
Epoch 4/20
522/522 [==============================] - 3s 6ms/step - loss: 1.1407 - mae: 0.6928
Epoch 5/20
522/522 [==============================] - 3s 6ms/step - loss: 1.1090 - mae: 0.6852
Epoch 6/20
522/522 [==============================] - 3s 6ms/step - loss: 1.0868 - mae: 0.6786
Epoch 7/20
522/522 [==============================] - 3s 6ms/step - loss: 1.0657 - mae: 0.6715
Epoch 8/20
522/522 [==============================] - 3s 6ms/step - loss: 1.0473 - mae: 0.6634
Epoch 9/20
522/522 [==============================] - 3s 6ms/step - loss: 1.0384 - mae: 0.6605
Epoch 10/20
522/522 [==============================] - 3s 6ms/step - loss: 1.0299 - mae: 0.6578
Epoch 11/20
522/522 [============================

In [11]:
results = pd.DataFrame(results, columns=["n_layers", "n_nodes", "activation", "time", "MSE", "MAE", "MAPE", "RMSE"])
results.sort_values(by="MAPE")

,n_layers,n_nodes,activation,time,MSE,MAE,MAPE,RMSE
14,2,468.0,relu,170.230150,0.855622,0.519494,0.337527,0.924998
5,2,234.0,relu,85.365216,0.857593,0.527868,0.342968,0.926063
13,2,468.0,sigmoid,177.045295,0.852878,0.536870,0.348817,0.923514
17,3,468.0,relu,250.275441,0.840547,0.538884,0.350126,0.916813
8,3,234.0,relu,102.028562,0.840395,0.550264,0.357519,0.916731
23,2,702.0,relu,285.557254,0.844407,0.551418,0.358269,0.918916
4,2,234.0,sigmoid,87.762864,0.844261,0.562799,0.365664,0.918837
16,3,468.0,sigmoid,258.835724,0.847704,0.570464,0.370643,0.920708
19,1,702.0,sigmoid,130.797832,0.921748,0.579098,0.376254,0.960077
1,1,234.0,sigmoid,63.501523,0.944158,0.579791,0.376704,0.971678
